In [ ]:
#구글 드라이브 디렉토리를 이용하기 위해 마운트 합니다
#이미 마운트 되어 있는 경우 실행하지 않아도 됩니다
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#필요한 라이브러리를 설치합니다
!pip install  mxnet
!pip install  gluonnlp pandas tqdm
!pip install  sentencepiece
!pip install  transformers==3 
!pip install  datasets
!pip install --target=$my_path git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 55.0MB 57kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 8.8MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588515 sha256=8a72de952007a636c3e219b56b6dde007e5525417c185e3b0cb95ca71cbff89b
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 8.7MB/s 
     |████████████████████████████████| 757kB 6.5MB/s 
     |████████████████████████████████| 3.0MB 20.1MB/s 
     |████████████████████████████████| 890kB 30.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a021f7e1f5e34ee033d00c02166a45b02c381993ea7c4d6b73c00a503fc3c99f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a5012

In [ ]:
#test 데이터를 미리 불러옵니다
import pandas as pd

test = pd.read_csv('/content/news_test.csv', encoding = 'utf-8-sig')

In [ ]:
#시간 측정 시작
import time
start = time.time()

## Libraray import및 토크나이저 생성

In [ ]:
#필요한 library들을 import 합니다
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm, tqdm_notebook

device = torch.device('cuda')

In [ ]:
#토크나이저를 생성합니다
import sys
sys.path.append('/content/drive/MyDrive/K-러닝/3.Tokenizer/DistilKoBERT-master')
from tokenization_kobert import KoBertTokenizer

tf_tok = KoBertTokenizer.from_pretrained('monologg/kobert')

## 모델 생성

In [ ]:
#모델 구조에 관한 자세한 설명은 설명 자료에 첨부되어있습니다
#KoBERT모델을 이용하여 custom classifier를 생성하여 예측하는 모델입니다
#classification층은 총 3층입니다 
from transformers import BertPreTrainedModel
from transformers import BertModel
from torch.nn import CrossEntropyLoss, MSELoss

class custom_bert_clf(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config) #KoBERT가 BERT 모델에 사용됩니다
        self.dropout = nn.Dropout(config.hidden_dropout_prob) 
        self.classifier = nn.Linear(config.hidden_size, 256) #분류를 수행하기 위한 classification층입니다. 
        self.classifier2 = nn.Linear(256, 64)
        self.classifier3 = nn.Linear(64, config.num_labels)
        self.gelu = nn.GELU() #classification 층에서 사용되는 활성함수입니다
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):


        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)

        cl1_output = self.classifier(pooled_output)
        cl1_output= self.gelu(cl1_output)
        cl1_output = self.dropout(cl1_output)

        cl2_output = self.classifier2(cl1_output)
        cl2_output = self.gelu(cl2_output)
        cl2_output = self.dropout(cl2_output)

        logits = self.classifier3(cl2_output)

        outputs = (logits,) + outputs[2:]  

        if labels is not None:
            if self.num_labels == 1:

                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  

In [ ]:
#위에서 선언한 class를 바탕으로 학습한 모델을 불러옵니다
tf_bertmodel = custom_bert_clf.from_pretrained('/content/drive/MyDrive/K-러닝/5.Model')

## 데이터 전처리 작업

In [ ]:
#전처리 시작1
#정규표현식들을 이용하여 기사내에 존재하는 일부 문자열들을 제거 및 변환하는 작업을 수행하였습니다
import re
import numpy as np

stopwords_kr = pd.read_csv('/content/drive/MyDrive/K-러닝/1.Data/stopwords_kr.txt',sep = '\n', header = None)
stop_kr_list = list(stopwords_kr.iloc[:,0])

url_with_https = re.compile(r'(https?:\/\/){1,}([\w.\/%-]+){1,}') #url 탐지
rm_email = re.compile(r'([a-zA-Z0-9\.\_]+)@([\w\.]+){1,}') #email 탐지
useless_words = re.compile(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]+') #특수문자 탐지
rm_float_num = re.compile(r'(\d+\.?\,?\d+)+') #소수점 형태로 입력된 숫자를 탐지
rm_num = re.compile(r'\d') #숫자를 탐지

def clean_kor_text(kor_text):
    text = kor_text.lower() #영어가 있는경우 대문자들을 모두 소문자로 변경하기
    text = url_with_https.sub('url',text) #영어 'url'이라고 일괄적으로 변경하기
    text = rm_email.sub('email', text) #영어 'email'이라고 일괄적으로 변경하기
    text = useless_words.sub('',text) #특수문자 제거
    text = rm_float_num.sub('',text) #소숫점 숫자 제거
    text = rm_num.sub('', text) #숫자 제거
    text = ' '.join(word for word in text.split() if word not in stop_kr_list) 
    return text


test_c_cont = test['content'].apply(lambda x: clean_kor_text(x))
test_c_title = test['title'].apply(lambda x: clean_kor_text(x) + ' ')

test_c_df = pd.concat([test_c_title + test_c_cont], axis = 1) #데이터 프레임으로 생성

In [ ]:
test_c_df.columns = ['tle_cont']
test_c_df['info'] = 0
test_c_df.to_csv('test_c_df.txt', index = False) #Dataset 형태로 만들기 위해 txt형태로 저장

## Dataset 및 Dataloader 객체 생성

In [ ]:
#pytorch에서 호환되는 형식의 Dataset class를 생성합니다
class nh_news_Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

In [ ]:
#위에서 생성한 pytorch형 Dataset 생성을 일원화 하는 함수를 선언하고 실행합니다
from sklearn.model_selection import train_test_split

def make_nh_dataset(file_path, tokenizer, eval = False):

  df = pd.read_csv(file_path)

  df_texts = df['tle_cont'].tolist()
  df_labels = df['info'].tolist()

  if eval == True:
    train_texts, val_texts, train_labels, val_labels = train_test_split(df_texts, df_labels,
                                                                    test_size = 0.1, random_state = 156,
                                                                    stratify = df_labels)
    
    train_encodings = tokenizer(train_texts, truncation= True, padding= True, max_length= 128) 
    val_encodings = tokenizer(val_texts, truncation= True, padding= True, max_length= 128)

    train_dataset = nh_news_Dataset(train_encodings, train_labels)
    val_dataset = nh_news_Dataset(val_encodings, val_labels)

    return train_dataset, val_dataset

  else:
    text_encodings = tokenizer(df_texts, truncation= True, padding= True, max_length= 128)

    dataset = nh_news_Dataset(text_encodings, df_labels)

    return dataset

In [ ]:
test_dataset = make_nh_dataset('test_c_df.txt', tf_tok)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size= 64, num_workers = 5, shuffle= False)

## 예측

In [ ]:
pred = []

tf_bertmodel.eval()

for batch_id, dataset in enumerate(tqdm_notebook(test_dataloader)):
  input_ids = dataset['input_ids'].to(device)
  attention_mask = dataset['attention_mask'].to(device)
  token_type_ids = dataset['token_type_ids'].to(device)

  tf_bertmodel.to(device)

  with torch.no_grad():
    out = tf_bertmodel(input_ids = input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)[0]

  logits = out.detach().cpu().numpy()
  pred.append(logits)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [ ]:
pred_result = []
for i in range(len(pred)):
  for j in range(len(pred[i])):
    max_vals, max_indices = torch.max(torch.Tensor(pred[i][j].reshape(1, -1)), 1)
    max_indices = max_indices.detach().cpu().numpy()
    pred_result.extend(max_indices)

In [ ]:
print(time.time() - start)

1136.791264295578


## csv 생성

In [ ]:
sub = pd.concat([test['id'], pd.Series(pred_result)], axis = 1)
sub.columns = ['id', 'info']

In [ ]:
sub.to_csv('/content/submission.csv', index = False) 